In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from keras.utils import image_dataset_from_directory
from keras.optimizers import Adam
from keras.activations import relu, swish
from keras.applications import (
    ResNet101 , ResNet152 , ResNet101V2 , ResNet50 ,
    EfficientNetB0 , EfficientNetB1 , EfficientNetB7 ,
    VGG16 , VGG19

)
from keras.applications.resnet import preprocess_input as preprocess_fn_res
from keras.applications.vgg16 import preprocess_input as preprocess_fn_vgg
from keras.applications.efficientnet import preprocess_input as preprocess_fn_eff
from keras.regularizers import l1,l2
from keras.layers import RandomBrightness , RandomContrast , RandomFlip , RandomZoom , GlobalAveragePooling2D , BatchNormalization , Dropout , Dense
import kagglehub
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam




# Download Data set from Kaggle

In [9]:
#pip install kagglehub

path = kagglehub.dataset_download("ananthu017/emotion-detection-fer")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'emotion-detection-fer' dataset.
Path to dataset files: /kaggle/input/emotion-detection-fer


# Set test and train path

In [11]:

train_path = path+"/train"
test_path = path+"/test"

# Load DataSet

In [21]:
train_ds_motion      =  image_dataset_from_directory(train_path, batch_size=32, shuffle= True, color_mode= 'grayscale',subset='training', validation_split= 0.2, seed=42, image_size=(224,224))
validation_ds_motion =  image_dataset_from_directory(train_path, batch_size= 32, color_mode= 'grayscale', subset= 'validation', validation_split= 0.2, seed= 42, image_size= (224,224))
test_ds_motion       =  image_dataset_from_directory(test_path, batch_size=32, shuffle=True, color_mode= 'grayscale', image_size=(224,224))

Found 28709 files belonging to 7 classes.
Using 22968 files for training.
Found 28709 files belonging to 7 classes.
Using 5741 files for validation.
Found 7178 files belonging to 7 classes.


In [22]:
number_of_classes = 7

# Data Augmentation

In [23]:
def get_data_augmentation():
    return Sequential([
        RandomBrightness(0.1),
        RandomZoom(0.1),
        RandomFlip("horizontal"),
        RandomContrast(0.1)
    ])

# Data Preprocessing

In [24]:
def preprocessing(img, lbl):
    img = tf.cast(img,tf.float32)
    img = preprocess_fn_eff(img)
    lbl = tf.one_hot(lbl, number_of_classes)

    return img, lbl

In [25]:
train_ds_motion         = train_ds_motion.map(preprocessing, num_parallel_calls = tf.data.AUTOTUNE)
validation_ds_motion    = validation_ds_motion.map(preprocessing, num_parallel_calls = tf.data.AUTOTUNE)
test_ds_motion          = test_ds_motion.map(preprocessing, num_parallel_calls = tf.data.AUTOTUNE)


In [26]:
def build_model(fine_tune_at = -30 ) :
 base_model = EfficientNetB0(include_top=False , weights=None , input_shape=(224,224,1))
 # Fine Tunning
 base_model.trainable = False
 if fine_tune_at is not None :
     base_model.trainable = True
     for layer in base_model.layers[:fine_tune_at]:
      layer.trainable = False

 input  = keras.layers.Input(shape= (224,224,1) )
 x = input
 # Applying Data Augmentation
 data_augmentation = get_data_augmentation()
 x = data_augmentation(x)

 x =  base_model(x)

 x = GlobalAveragePooling2D() (x)
 x = BatchNormalization()(x)
 x = Dropout(0.3)(x)
 x = Dense(512 , activation="relu" , kernel_regularizer= l2(0.01))(x)
 x = BatchNormalization() (x)
 x = Dropout(0.3)(x)
 output = Dense(7 , activation="softmax")(x)
 model = keras.Model(input , output)
 return model

In [27]:
model = build_model()
model.compile(loss =  "categorical_crossentropy" , optimizer = Adam(0.001) , metrics = ["accuracy"] )

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,048,991 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,715,622 (17.99 MB)

 Trainable params: 2,159,207 (8.24 MB)

 Non-trainable params: 2,556,415 (9.75 MB)

In [30]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(
    "best_emotion_model.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-7, verbose=1)

In [ ]:
history = model.fit(
    train_ds_motion,
    epochs=30,
    validation_data=validation_ds_motion,
    callbacks=[early_stopping, checkpoint, reduce_lr]
)


Epoch 1/30
718/718 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.2640 - loss: 2.1441
Epoch 1: val_accuracy improved from -inf to 0.17715, saving model to best_emotion_model.keras
718/718 ━━━━━━━━━━━━━━━━━━━━ 85s 118ms/step - accuracy: 0.2640 - loss: 2.1441 - val_accuracy: 0.1771 - val_loss: 2.1193 - learning_rate: 0.0010
Epoch 2/30
 60/718 ━━━━━━━━━━━━━━━━━━━━ 1:01 93ms/step - accuracy: 0.2700 - loss: 2.0479